# Data Analysis: Education, Literacy, and UN Speech Patterns in Africa

## Research Questions

### 1- Exploratory Research Question:
Is there a correlation between the frequency of education-related words in African country speeches and literacy rates?

### 2- Predictive Research Question:
Can we predict literacy rates using education expenditure and the frequency of education-related words in a multi-regression model? **Change**

## Hypotheses

- There is a significant correlation between education word frequency and literacy rates.
- CHANGE: **Not sure yet, we need to do some research before this!**


## Methodology Overview

1. Data Integration: Combine UN speech data, literacy rates, and education expenditure data for African countries
2. Text Processing: Extract and count education-related terms from UN speeches
3. Exploratory Analysis: Examine relationships between variables
4. Statistical Analysis: Investigate correlations and regression modeling


## 1. Data Loading and Initial Setup


In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import scipy.stats as stats
import unicodedata
import re

# If needed, install the requirements
#!pip install -r requirements.txt



### 1.1 Load UN Speeches Data


In [ ]:
# Load UN Speeches from text files
sessions = np.arange(1, 80) # include all sessions
data = []

for session in sessions:
    directory = "./Data/TXT/Session "+str(str(session).zfill(2))+" - "+str(1945+session)
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename)) as f:
            if filename[0]==".": #ignore hidden files
                continue
            splt = filename.split("_")
            data.append([session, 1945+session, splt[0], f.read()])

# Create DataFrame and extract fundamental information about the data
df_speech = pd.DataFrame(data, columns=['Session', 'Year', 'ISO-alpha3 Code', 'Speech'])
print(f"Loaded {len(df_speech)} speeches from {df_speech['Year'].nunique()} years")
print(f"Year range: {df_speech['Year'].min()}-{df_speech['Year'].max()}")
print(f"Number of unique countries: {df_speech['ISO-alpha3 Code'].nunique()}")

# Check the dataframe
df_speech.tail(20)
#df_speech.head(20)


### 1.2 Load Country Codes and Regional Information


In [ ]:
# Load country codes and regional information
codes_path = "./Data/UNSD_Methodology.csv"
df_codes = pd.read_csv(codes_path, sep=";")

# Extract fundamental information about the data
print(f"Country codes dataset shape: {df_codes.shape}")
print(f"Regions available: {df_codes['Region Name'].unique()}")

# Check the dataframe
df_codes.head(20)
#df_codes.tail(20)

Merging the UN Speeches with Region-Country Information (Methodology.csv)

In [ ]:
# Merge speeches with country information
df_un_merged = pd.merge(
    df_speech[["ISO-alpha3 Code", "Session", "Year", "Speech"]],
    df_codes[["Country or Area", "Region Name", "ISO-alpha3 Code"]],
    on="ISO-alpha3 Code",
    how="left"
)

# Print fundamental information about the merged dataframe
print(f"Merged dataset shape: {df_un_merged.shape}")

# Print the unique countries without the region name
print("All rows with missing Region Name:")
print(df_un_merged[df_un_merged["Region Name"].isna()])

print("Unique countries/entities with missing Region Name:")
print(df_un_merged[df_un_merged["Region Name"].isna()][["ISO-alpha3 Code", "Country or Area"]].drop_duplicates()) # None of them in Africa, so skip.

# Check the dataframe
df_un_merged.head(20) # or df_un_merged.tail(20)

### 1.3 Extract Countries within the Africa Region


In [ ]:
# Filter for African countries
df_africa = df_un_merged[df_un_merged["Region Name"] == "Africa"].copy()

# Print fundamental information about the dataframe
print(f"African countries dataset shape: {df_africa.shape}")
print(f"\nNumber of African countries: {df_africa['Country or Area'].nunique()}")
print(f"\nYear range for African speeches: {df_africa['Year'].min()}-{df_africa['Year'].max()}")
print(f"\nAfrican countries in dataset:\n{df_africa['Country or Area'].unique()}")
print(f"\nMissing values per column:\n{df_africa.isna().sum()}")

# Search for non-ASCII characters
for col in ["Speech"]:
    non_ascii_rows = []
    for idx, val in df_africa[col].astype(str).items():
        for ch in val:
            if not ch.isascii():
                non_ascii_rows.append(idx)
                break
    print(f"\nColumn: {col}")
    print(f"Rows with non-ASCII characters: {len(non_ascii_rows)}")
    if non_ascii_rows:
        print(df_africa.loc[non_ascii_rows, [col, "Year"]].head(10))

# Check the dataframe
df_africa.head(20)
df_africa.tail(20)

### Clean & Preprocess the Africa Dataset

In [ ]:
# Define a pre-process function to normalize all text to lowercase, whitespace between, no special characters etc.
def preprocess_text(s: str) -> str:
    if pd.isna(s):
        return s
    # normalize unicode accents → ascii
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")

    # lowercase
    s = s.lower()

    # remove markdown bold/italics markers (i.e., ** and *)
    s = re.sub(r"\*+", "", s)

    # replace newlines/tabs with space
    s = re.sub(r"[\n\t\r]+", " ", s)

    # collapse multiple spaces into one
    s = re.sub(r"\s+", " ", s)

    # strip leading/trailing spaces
    return s.strip()

# Apply to Speech
df_africa["Speech"] = df_africa["Speech"].apply(preprocess_text)

# Check the dataframe
df_africa.head(20) # or df_africa.tail(20)


### 1.4 Load Literacy Rates Data


In [ ]:
# Load literacy rates dataset
df_literacy = pd.read_csv("Data/WB_GS_SE_ADT.csv")

# Print fundamental information about the data
print(f"Literacy dataset shape: {df_literacy.shape}")
print(f"Year range: {df_literacy['TIME_PERIOD'].min()}-{df_literacy['TIME_PERIOD'].max()}")
print(f"Missing values: {df_literacy['OBS_VALUE'].isna().mean()*100:.1f}%")
print(f"Number of countries/regions: {df_literacy['REF_AREA_LABEL'].nunique()}\n")
print(df_literacy.info())     # column names + types
print("\nCountries/Regions:", df_literacy["REF_AREA_LABEL"].nunique())
print(df_literacy["REF_AREA_LABEL"].unique()[:10])   # first 10 countries

print("\nSex categories:", df_literacy["SEX_LABEL"].unique())
print("\nAge groups:", df_literacy["AGE_LABEL"].unique())
print("\nUrbanisation categories:", df_literacy["URBANISATION_LABEL"].unique())
print("\nIndicators:", df_literacy["INDICATOR_LABEL"].unique())

#TODO: preprocess if needed?

# Check the dataframe
df_literacy.head(20)

### 1.5 Load Education Expenditure Data


In [ ]:
# Load education expenditure dataset (Government expenditure on education as % of GDP)
df_edu_exp = pd.read_csv("Data/UIS_EDSTATS_XGDP_FSGOV.csv")

# Print fundamental information about the data
print(f"Education expenditure dataset shape: {df_edu_exp.shape}")
print(f"Year range: {df_edu_exp['TIME_PERIOD'].min()}-{df_edu_exp['TIME_PERIOD'].max()}")
print(f"Missing values: {df_edu_exp['OBS_VALUE'].isna().mean()*100:.1f}%")
print(f"Number of countries: {df_edu_exp['REF_AREA_LABEL'].nunique()}")
print(f"\nExpenditure statistics:")
print(df_edu_exp['OBS_VALUE'].describe())

# Check the dataframe
df_edu_exp.head(20)
#TODO: preprocess this dataset as well. Check out missing, non-ascii, etc. 


## 2. Education-Related Terms Extraction

We extract education-related terms from the UNESCO thesaurus to identify relevant vocabulary in UN speeches.


In [ ]:
# Load education-related terms from UNESCO thesaurus
with open("./Data/sh85040989.json", "r", encoding="utf-8") as f:
    education_glossary_data = json.load(f)

# Collect all the @value entries from the JSON, as education-related words are encoded within.
education_related_terms = []
for entry in education_glossary_data:
    for key, values in entry.items():
        if isinstance(values, list):
            for value in values:
                if isinstance(value, dict) and "@value" in value:
                    education_related_terms.append(value["@value"])

# Create DataFrame
df_education_terms = pd.DataFrame(education_related_terms, columns=["Education-related Key Word"])
print(f"Initial education terms extracted: {len(df_education_terms)}")
df_education_terms.head(20)


### 2.1 Clean & Preprocess Education Terms Dataset

In [ ]:
# Clean and preprocess education terms
df_education_terms_clean = df_education_terms.drop_duplicates()
print(f"After removing duplicates: {len(df_education_terms_clean)}")

# Remove entries that start with digits (library classification codes)
df_education_terms_clean = df_education_terms_clean[
    ~df_education_terms_clean["Education-related Key Word"].str[0].str.isdigit()
]
print(f"After removing entries starting with digits: {len(df_education_terms_clean)}")

# Replace double dashes with spaces
df_education_terms_clean["Education-related Key Word"] = df_education_terms_clean[
    "Education-related Key Word"].str.replace("--", " ")

# Normalize whitespaces and convert to lowercase
df_education_terms_clean["Education-related Key Word"] = df_education_terms_clean[
    "Education-related Key Word"].str.replace(r"\s+", " ", regex=True).str.strip().str.lower()

# Ensure ASCII-only characters
df_education_terms_clean["Education-related Key Word"] = df_education_terms_clean[
    "Education-related Key Word"].str.encode('ascii', 'ignore').str.decode('ascii')

print(f"Final cleaned education terms: {len(df_education_terms_clean)}")
print("\nSample cleaned terms:")
print(df_education_terms_clean.head(15))

#TODO: using the nltk library, check out the distribution of the education-related terms.


## 3. Education-Related Word Frequency Calculation

### 3.1 Methodology for Frequency Calculation

This section will calculate the frequency of education-related words in African country speeches. The methodology involves:

1. Text Preprocessing: 
   - Tokenize each speech into individual words
   - Convert to lowercase for case-insensitive matching
   - Remove punctuation and special characters
   - Remove stop words (common words like "the", "and", "of")

2. Term Matching:
   - For each speech, count occurrences of each education-related term from our cleaned vocabulary
   - Handle both exact matches and partial matches (e.g., "education" should match "educational")
   - Consider word boundaries to avoid false matches

3. Frequency Metrics:
   - Absolute Count: Total number of education-related words per speech
   - Relative Frequency: Education words as percentage of total words in speech
   - Normalized Score: Account for speech length variations
   - Term Diversity: Number of unique education terms used

4. Aggregation Strategies:
   - Country-Year Level: Sum frequencies for each country per year
   - Country Level: Average frequencies across all years for each country
   - Temporal Analysis: Track frequency changes over time

### 3.2 Implementation

Expected Output: A new column in `df_africa` containing education word frequencies for each speech, which will enable correlation analysis with literacy rates.


In [ ]:
# Download required NLTK libraries
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

In [ ]:
# Prepare the list of education-related terms (lowercase, unique)
education_terms_list = df_education_terms_clean["Education-related Key Word"].tolist()

# Preprocess education terms for partial matching (e.g., "education" matches "educational")
# We'll use set of all words in all terms for partial matching
edu_terms_flat = set()
for term in education_terms_list:
    for word in term.split():
        edu_terms_flat.add(word)

# Prepare stopwords and punctuation
stop_words = set(stopwords.words('english'))
punct_table = str.maketrans('', '', string.punctuation)

def preprocess_text(text):
    text = text.translate(punct_table).lower()
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and w.isalpha()]
    return tokens

def count_education_terms(text, edu_terms_set):
    tokens = preprocess_text(text)
    total_words = len(tokens)
    
    # Count education-related words (partial match: if token starts with any edu term)
    edu_word_count = 0
    matched_terms = set()
    for token in tokens:
        for edu_term in edu_terms_set:
            if token.startswith(edu_term):
                edu_word_count += 1
                matched_terms.add(edu_term)
                break

    # Frequency as percentage
    edu_frequency_pct = (edu_word_count / total_words * 100) if total_words > 0 else 0

    # Diversity: number of unique education terms matched
    edu_term_diversity = len(matched_terms)

    return pd.Series([edu_word_count, total_words, edu_frequency_pct, edu_term_diversity])

# Apply to df_africa
df_africa[['edu_word_count', 'total_words', 'edu_frequency_pct', 'edu_term_diversity']] = df_africa['Speech'].apply(
    lambda x: count_education_terms(x, edu_terms_flat)
)


In [ ]:
# Check the dataframe
df_africa.head(20)
#df_africa.tail(20)

#TODO: Check the education related words found in speeches

## 4. Data Preparation and Integration

### 4.1 Prepare Datasets for Analysis

Before conducting our exploratory analysis, we need to integrate the three main datasets:
1. African UN speeches with education word frequencies
2. Literacy rates for African countries
3. Education expenditure data for African countries


In [ ]:
# Prepare literacy data for African countries
# Filter for total population (not gender-specific) and 15+ age group
df_literacy_clean = df_literacy[
    (df_literacy['SEX_LABEL'] == 'Total') & 
    (df_literacy['AGE_LABEL'] == '15 years old and over') &
    (df_literacy['OBS_VALUE'].notna())
].copy()

# Rename columns for clarity
df_literacy_clean = df_literacy_clean.rename(columns={
    'REF_AREA_LABEL': 'Country',
    'TIME_PERIOD': 'Year', 
    'OBS_VALUE': 'Literacy_Rate'
})

# Extract size information about the dataframe
print(f"Literacy data after filtering: {len(df_literacy_clean)} rows")
print(f"African countries in literacy data: {df_literacy_clean['Country'].unique().size}")

# Check which African countries have literacy data
african_countries_in_literacy = set(df_literacy_clean['Country'].unique())
african_countries_in_speeches = set(df_africa['Country or Area'].unique())

# Check overlap between African countries in literacy and speech datasets to determine which countries are eligible for data analysis
print(f"\nCountries that are present in both datasets: {len(african_countries_in_literacy & african_countries_in_speeches)}")
print("Eligible countries for data analysis:", african_countries_in_literacy & african_countries_in_speeches)


In [ ]:
# Prepare education expenditure data
df_edu_exp_clean = df_edu_exp[df_edu_exp['OBS_VALUE'].notna()].copy()

# Rename columns for clarity
df_edu_exp_clean = df_edu_exp_clean.rename(columns={
    'REF_AREA_LABEL': 'Country',
    'TIME_PERIOD': 'Year',
    'OBS_VALUE': 'Education_Expenditure_GDP_Pct'
})

print(f"Education expenditure data: {len(df_edu_exp_clean)} rows")
print(f"Countries in expenditure data: {df_edu_exp_clean['Country'].nunique()}")

# Check which African countries have expenditure data
african_countries_in_expenditure = set(df_edu_exp_clean['Country'].unique())

# Check overlap between African countries in literacy, speech, and expenditure datasets
common_countries_all = african_countries_in_literacy & african_countries_in_speeches & african_countries_in_expenditure

print(f"\nCountries in all three datasets: {len(common_countries_all)}")
print("Data Analysis-eligible Countries with complete data:", common_countries_all)


## 5. Exploratory Data Analysis

### 5.1 UN Speech Data Overview (African Countries)


In [ ]:
# Analyze UN speech data for African countries
print(f"African UN Speech Data Overview:".ljust(40), end="  ")
print(f"Total speeches: {len(df_africa)}")
print(f"Countries: {df_africa['Country or Area'].unique()}")
print(f"Year range: {df_africa['Year'].min()}-{df_africa['Year'].max()}")

# Speeches per country
speeches_per_country = df_africa['Country or Area'].value_counts()
print(f"\nSpeeches per country (top 20):".ljust(40), end="  ")
print(speeches_per_country.head(20).to_string())

# Speech lengths (character count)
df_africa['Speech_Length'] = df_africa['Speech'].str.len()
print(f"\nSpeech length statistics (characters):".ljust(40), end="  ")
print(df_africa['Speech_Length'].describe().to_string())

# Adjust the plot size for better readability
plt.figure(figsize=(12, 5))

# Number of speeches by country (Top 20) and Speech length distribution
plt.subplot(1, 2, 1)
speeches_per_country.head(20).plot.bar(color='green')
plt.title('Speeches by Country (Top 20)')
plt.xlabel('Country')
plt.ylabel('Number of Speeches')
plt.xticks(rotation=45, ha='right')

plt.subplot(1, 2, 2)
plt.hist(df_africa['Speech_Length'], bins=20, edgecolor='black', color='green')
plt.title('Speech Length Distribution')
plt.xlabel('Speech Length (characters)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Line plot: Number of speeches per year
speeches_by_year = df_africa.groupby('Year').size()
plt.plot(speeches_by_year.index, speeches_by_year.values, marker='o', color='green')
plt.title('Number of Speeches by Year')
plt.xlabel('Year')
plt.ylabel('Number of Speeches')
plt.tight_layout()
plt.show()


### 5.2 Literacy Rates Data Overview


In [ ]:
# Analyze literacy rates for African countries
african_literacy = df_literacy_clean[df_literacy_clean['Country'].isin(african_countries_in_speeches)]

# Extract descriptive statistics about the dataframe
print("Literacy Rate Statistics for African Countries:")
print(african_literacy['Literacy_Rate'].describe())

# Compute literacy rates over time
literacy_over_time = african_literacy.groupby('Year')['Literacy_Rate'].mean().reset_index()

# Adjust the plot size for better readability
plt.figure(figsize=(12, 5))

# Distribution of all literacy rates within African countries
plt.subplot(1, 2, 1)
plt.hist(african_literacy['Literacy_Rate'], bins=20, color='green', edgecolor='black')
plt.title('Literacy Rate Distribution Within African Countries')
plt.xlabel('Literacy Rate (%)')
plt.ylabel('Count')

# Literacy rates over time within African countries
plt.subplot(1, 2, 2)
plt.plot(literacy_over_time['Year'], literacy_over_time['Literacy_Rate'], marker='o', color='green')
plt.title('Mean Literacy Rate Over Time Within African Countries')
plt.xlabel('Year')
plt.ylabel('Mean Literacy Rate (%)')

# Print the plots
plt.show()



### 5.3 Education Expenditure in African Countries

**CHANGE For the predictive analysis, we need to have a dataset that has the education expenditure data for all African countries for all years.**


In [ ]:
#TODO: for the predictive analysis, we need to have a dataset that has the education expenditure data for all African countries for all years.

## 6. Research Question Analysis

### 6.1 Exploratory Question: Correlation Between Education Words and Literacy Rates

The analysis will explore:
1. **Country-level correlation based on frequency percentage**: Average education word frequency vs. literacy rates
2. **Country-level correlation based on total word countn**: Total education-related word vs literacy rates

Methods:
- Pearson correlation coefficient
- Scatter plots
- Statistical significance testing
- Outlier analysis


In [ ]:
# Correlation analysis between education word frequency and literacy rates

# --- Prepare data ---

# Calculate mean education word frequency and total education words per country
edu_stats = df_africa.groupby('Country or Area').agg({
    'edu_frequency_pct': 'mean',
    'total_words': 'mean'
}).reset_index()

# Get latest literacy rate per country
latest_lit = df_literacy_clean.sort_values('Year').groupby('Country').last().reset_index()

# Merge for frequency analysis
merged_freq = pd.merge(
    edu_stats[['Country or Area', 'edu_frequency_pct']],
    latest_lit[['Country', 'Literacy_Rate']],
    left_on='Country or Area',
    right_on='Country'
)

# Merge for total words analysis
merged_total = pd.merge(
    edu_stats[['Country or Area', 'total_words']],
    latest_lit[['Country', 'Literacy_Rate']],
    left_on='Country or Area',
    right_on='Country'
)

# Statistical analysis: Education word frequency (%) vs Literacy Rate

# Pearson correlation
corr_freq, p_freq = stats.pearsonr(merged_freq['edu_frequency_pct'], merged_freq['Literacy_Rate'])
print(f"Pearson correlation (frequency %): {corr_freq:.3f} (p={p_freq:.3g})")

# Paired t-test (not ideal for correlation, but included for comparison)
t_stat_freq, p_value_freq = stats.ttest_rel(merged_freq['edu_frequency_pct'], merged_freq['Literacy_Rate'])
print(f"T-statistic (frequency %): {t_stat_freq:.3f}, p-value: {p_value_freq:.3g}")

# Statistical analysis: Total education words vs Literacy Rate

# Pearson correlation
corr_total, p_total = stats.pearsonr(merged_total['total_words'], merged_total['Literacy_Rate'])
print(f"Pearson correlation (total words): {corr_total:.3f} (p={p_total:.3g})")

# Paired t-test (not ideal for correlation, but included for comparison)
t_stat_total, p_value_total = stats.ttest_rel(merged_total['total_words'], merged_total['Literacy_Rate'])
print(f"T-statistic (total words): {t_stat_total:.3f}, p-value: {p_value_total:.3g}")

# --- Plots ---

# Scatter plot: Education word frequency (%) vs Literacy Rate
plt.scatter(merged_freq['edu_frequency_pct'], merged_freq['Literacy_Rate'])
z_freq = np.polyfit(merged_freq['edu_frequency_pct'], merged_freq['Literacy_Rate'], 1)
plt.plot(merged_freq['edu_frequency_pct'], np.polyval(z_freq, merged_freq['edu_frequency_pct']), "r--")
plt.xlabel('Education Word Frequency (%)')
plt.ylabel('Literacy Rate (%)')
plt.title('Education Word Frequency vs Literacy Rate')
plt.show()

# Scatter plot: Total education words vs Literacy Rate
plt.scatter(merged_total['total_words'], merged_total['Literacy_Rate'])
if merged_total['total_words'].nunique() > 1:
    z_total = np.polyfit(merged_total['total_words'], merged_total['Literacy_Rate'], 1)
    plt.plot(
        merged_total['total_words'],
        np.polyval(z_total, merged_total['total_words']),
        "r--"
    )
plt.xlabel('Total Education Words (mean per country)')
plt.ylabel('Literacy Rate (%)')
plt.title('Total Education Words vs Literacy Rate')
plt.show()

# --- Summary statistics ---
print("Summary statistics for education word frequency (%) and literacy rate:")
print(merged_freq[['edu_frequency_pct', 'Literacy_Rate']].describe())
print("\nSummary statistics for total education words and literacy rate:")
print(merged_total[['total_words', 'Literacy_Rate']].describe())

# TODO: Outlier analysis  



### 6.2 Predictive Question: <>

